In [ ]:
from modules.module_for_imports import *

# Inconsistent values removal
> ### In this file we will find out how to remove all the elements we don't want in the SQL server.
1. We'll clean up the players, checking the **current_club_id** column
2. We'll clean up BOTH **game_events** and **club_games**, checking both the foreign keys from each set
    - ***game_events***: player_id and game_id
    - ***club_games***: game_id and club_id
        
3. After that, we will analyse **appearances**, **competitions** and **game_lineups**, to avoid issues with the retrieve of the GET routes.
---
## Results:
For points 1. and 2.:
    - `game_events.player_id` data are **NOT** consistent
    - `club_games.club_id` data are **NOT** consistent -> we will have to remove also the other tuple with the same `game_id`
    - All other data checked are consistent
For point 3:
    - `competitions` has a **useless** competition - Fifa Klub wm - which has no game linked to it.
    - `appearances` has all player_id and competition_id references, but 70,2k of `game_id`'s are not consistent with `games` table. Furthermore, 7,2k `club_id` foreign_keys have not a match in the dataset (but they are part of the 70k we will remove to maintain `game_id` consistency).
    - `game_lineups` we found 41k inconsistent rows where `game_id` has no match. `club_id` with no match are in this subset, so we can delete the initial 41k and that's all we have to do.  
    

---
# Defining the functions used in this section

In [ ]:
def check_consistency(df1, df2, column_name1, column_name2):
    result = df1[df1[column_name1].isin(df2[column_name2])]
    print('Modified.' if (result.shape[0] != df1.shape[0]) else 'Not modified.')
    return result

---
# 1. Cleaning players 
- For now, players seems to have all `current_club_id` consistent with clubs table.

In [ ]:
# locate = '../' # usable location vrb 
games = clean_games(get_games('../'))
clubs = clean_clubs(get_clubs('../'))
#players will be modified!
players = clean_players(get_players('../'))

In [ ]:
players.info()

In [ ]:
player = check_consistency(players, clubs, 'current_club_id', 'club_id') # Not modified.

---
# 2.a Cleaning game_events
- `player_id` in game_events will be **not** consistent with players table.
- `game_id` is consistent with the games table.

In [ ]:
game_events = clean_game_events(get_game_events('../'))
game_events.info()

In [ ]:
ge_consistency = check_consistency(game_events, players, 'player_id', 'player_id') # Modified.
neg_game_events = game_events.query('not player_id.isin(@ge_consistency["player_id"])', engine='python')
print('Inconsistent rows:', neg_game_events.shape[0], 'of', game_events.shape[0])

In [ ]:
# It seems that there are player_id's in game_events that are not in players:
game_events[~game_events['player_id'].isin(players['player_id'])]

In [ ]:
players = None

In [ ]:
game_events = check_consistency(game_events, games, 'game_id', 'game_id') # Not modified.
game_events.shape

In [ ]:
game_events = None

---
# 2.b Cleaning club_games... 
- `game_id` is consistent with games table
- `club_id` is **NOT** consistent with clubs table

In [ ]:
club_games = clean_club_games(get_club_games('../'), '../')
club_games.info()

In [ ]:
games.query('game_id.isin(@club_games["game_id"])', engine='python').shape[0] == games.shape[0]

In [ ]:
check_consistency(club_games, games, 'game_id', 'game_id').head() # Not modified.

In [ ]:
cg_consistency = check_consistency(club_games, clubs, 'club_id', 'club_id') # Modified.

In [ ]:
neg_club_games = club_games.query('not club_id.isin(@cg_consistency["club_id"])', engine='python')
print('Inconsistent rows:', neg_club_games.shape[0], 'of', club_games.shape[0])

In [ ]:
club_games.query('game_id.isin(@neg_club_games["game_id"])', engine='python').shape[0] 

In [ ]:
games = None
clubs = None
club_games = None

# Order of removal:
1. `game_id` from **club_games** where `club_id` is not found in **clubs**
2. `game_id` from **games** according to **club_games** foreign_keys
3. `game_id` from **game_events** according to **games** foreign_keys
4. `player_id` from **game_events** according to **players** foreign_keys

---
# Making Appearances check

In [ ]:
loc = '../'
# clubs_n_games = clean_club_games(get_club_games(loc), loc)
clubs_n_games = clean_club_games(get_club_games(loc), games, loc) # defined again after this study 
games = clubs_n_games[1]
clubs = clubs_n_games[0]
app = clean_appearances(get_appearances(loc), loc)
players = clean_players(get_players(loc))
clubs_n_games = None
app.info()

In [ ]:
print(app.shape[0])
appea2 = check_consistency(app, games, 'game_id', 'game_id') # Modified: 70k extra rows
print(appea2.shape[0])

In [ ]:
check_consistency(app, players, 'player_id', 'player_id').shape[0] # Not modified.

In [ ]:
check_consistency(app, clubs, 'player_club_id', 'club_id').shape[0] # Modified: 7,2k

In [ ]:
check_consistency(appea2, clubs, 'player_club_id', 'club_id').shape[0]

In [ ]:
competitions = clean_competitions(get_competitions(loc))
check_consistency(app, competitions, 'competition_id', 'competition_id').shape[0] # Not modified

## ^^ We should remove:
 - 70,2k tuples for game_id consistency
 - 7.2k tuples for club_id consistency
<br>

---
<br>

# 'Competitions' check:
#### we should check the competition with `id=KLUB`.

In [ ]:
loc = '../'
competitions = clean_competitions(get_competitions(loc))
# games already present from appearances check up above.
print(competitions.info())
competitions.query('competition_id == "KLUB"')

In [ ]:
result = check_consistency(competitions, games, 'competition_id', 'competition_id')
result

In [ ]:
competitions[~competitions['competition_id'].isin(result['competition_id'])]

# We **have to remove** Fifa-klub-wm 
## -> There is no game of it, we can show nothing about it!

---
# 'Game_lineups' check
Firstly, we check `game_id` and `club_id`. After that, `player_id` and `player_name`.

In [ ]:
game_lu = clean_game_lineups(get_game_lineups(loc))
print(game_lu.info())
print(game_lu.shape[0])
game_lu2 = check_consistency(game_lu, games, 'game_id', 'game_id')  # Modified: 41.8k
game_lu2.shape[0]

In [ ]:
print(game_lu2.shape[0])
game_lu2 = check_consistency(game_lu2, clubs, 'club_id', 'club_id')     # Not modified.
game_lu2.shape[0]

### Inconsistent game ids have been found. If we clean them, `clubs` are ok.

In [ ]:
game_lu_players = check_consistency(game_lu2, players, 'player_id', 'player_id') # Modified: 1.6k
game_lu_players.shape[0]

In [ ]:
game_lu_players['player_name'].isna().sum()

In [ ]:
game_lu_to_remove = game_lu.drop(game_lu2.index)
print(game_lu_to_remove.shape[0])
game_lu_to_remove.query('type == "starting_lineup"').value_counts(['game_id'])  
# these are the game_lineups to remove, grouped by their starting_lineups